## Save listing contents as text
Save the contents of each listing (house or apartment) into a text file, to be used later by BeautifulSoup.

In [1]:
# Import necessary libraries
from selenium import webdriver
from bs4 import BeautifulSoup
import os
import time
import random
import re
from queue import Queue
from threading import Thread
import requests

In [2]:
# Create a folder for data
data_folder_path = os.path.abspath("./data/")
if not os.path.exists(data_folder_path):
    os.makedirs(data_folder_path)

In [3]:
# Folder for page contents
contents_folder_path = os.path.join(data_folder_path,"contents/")
if not os.path.exists(contents_folder_path):
    os.makedirs(contents_folder_path)

In [4]:
# The txt file for the links of the individual listings
listings_url_file_name = "listings_url_list_processed.txt" 
listings_url_file_path = os.path.join(data_folder_path,listings_url_file_name)

In [5]:
# Load links of listings
with open(listings_url_file_path, "r+") as file:
    urls = file.readlines()
print(urls)

['https://www.immoweb.be/en/classified/apartment/for-sale/ixelles/1050/8956021\n', 'https://www.immoweb.be/en/classified/house/for-sale/denderleeuw/9470/9528614\n', 'https://www.immoweb.be/en/classified/duplex/for-sale/geraardsbergen/9500/9555271\n', 'https://www.immoweb.be/en/classified/house/for-sale/uccle/1180/9064209\n', 'https://www.immoweb.be/en/classified/house/for-sale/anderlecht/1070/9391667\n', 'https://www.immoweb.be/en/classified/duplex/for-sale/arlon/6700/9551192\n', 'https://www.immoweb.be/en/classified/house/for-sale/bredene/8450/9559175\n', 'https://www.immoweb.be/en/classified/villa/for-sale/nivelles/1400/9451348\n', 'https://www.immoweb.be/en/classified/apartment/for-sale/uccle/1180/9317406\n', 'https://www.immoweb.be/en/classified/house/for-sale/woluwe-saint-pierre/1150/9406426\n', 'https://www.immoweb.be/en/classified/house/for-sale/limal-wavre/1300/9478369\n', 'https://www.immoweb.be/en/classified/house/for-sale/wangenies/6220/9419687\n', 'https://www.immoweb.be/en

In [6]:
print(len(urls))

9306


In [11]:
# For a single entry, get contents as a string
#set up the queue to hold all the urls
q = Queue(maxsize=0)
# Use many threads (50 max, or one for each url)
num_theads = min(20, len(urls))

In [12]:
# Populating Queue with tasks
# Specify the urls to run
range_url = range(0,min(len(urls), len(urls)))  # Change to run for each person

In [14]:
print(range_url)

range(0, 9306)


In [15]:
#Populating Queue with tasks
results = [{} for x in urls];
# load up the queue with the urls to fetch and the index for each job (as a tuple):
for i in range_url:
    #need the index and the url in each queue item.
    q.put((i,urls[i],contents_folder_path))

In [16]:
print(q.queue)

deque([(0, 'https://www.immoweb.be/en/classified/apartment/for-sale/ixelles/1050/8956021\n', 'C:\\Users\\ecebo\\MyRepos\\challenge-collecting-data\\data\\contents/'), (1, 'https://www.immoweb.be/en/classified/house/for-sale/denderleeuw/9470/9528614\n', 'C:\\Users\\ecebo\\MyRepos\\challenge-collecting-data\\data\\contents/'), (2, 'https://www.immoweb.be/en/classified/duplex/for-sale/geraardsbergen/9500/9555271\n', 'C:\\Users\\ecebo\\MyRepos\\challenge-collecting-data\\data\\contents/'), (3, 'https://www.immoweb.be/en/classified/house/for-sale/uccle/1180/9064209\n', 'C:\\Users\\ecebo\\MyRepos\\challenge-collecting-data\\data\\contents/'), (4, 'https://www.immoweb.be/en/classified/house/for-sale/anderlecht/1070/9391667\n', 'C:\\Users\\ecebo\\MyRepos\\challenge-collecting-data\\data\\contents/'), (5, 'https://www.immoweb.be/en/classified/duplex/for-sale/arlon/6700/9551192\n', 'C:\\Users\\ecebo\\MyRepos\\challenge-collecting-data\\data\\contents/'), (6, 'https://www.immoweb.be/en/classified

In [17]:
def save_contents_to_text(q, result):
    while not q.empty():
        work = q.get()  #fetch new work from the Queue
        #try:
        url = work[1]
        folder_path = work[2]
        print(folder_path)
        m = re.search("/(?P<immoweb_code>[\w\-]+)$", url)
        immoweb_code = m.group("immoweb_code")
            
        file_name = f"Contents_{immoweb_code}.txt"
        filepath = os.path.join(folder_path,file_name)
        print(filepath)
        # Check if the text file already exists
        if os.path.exists(filepath):
            print("File already exists")
            # return True
            
        print(url)
        r = requests.get(url)
        time.sleep(random.uniform(3.0, 5.0))
        print("Made request")
        # Display the requested url and the return of the server
        print(url, r.status_code)
        # Store the HTML content of the website in a text
        page_contents = r.text
        print(len(page_contents))
        
        with open(filepath,"w", encoding="utf-8") as input_file:
            input_file.write(page_contents)

        print("Written file ..." + immoweb_code)
            
             # Save file to text
        #except:
            #print('Error with URL check!')
        # Signal to the queue that task has been processed
        q.task_done()
    return True

In [18]:
# Starting worker threads on queue processing
for i in range(num_theads):
    print('Starting thread ', i)
    worker = Thread(target=save_contents_to_text, args=(q,results))
    worker.setDaemon(True)    #setting threads as "daemon" allows main program to 
                              #exit eventually even if these dont finish 
                              #correctly.
    worker.start()

Starting thread  0
C:\Users\ecebo\MyRepos\challenge-collecting-data\data\contents/
C:\Users\ecebo\MyRepos\challenge-collecting-data\data\contents/Contents_8956021.txt
Starting thread https://www.immoweb.be/en/classified/apartment/for-sale/ixelles/1050/8956021

 1
C:\Users\ecebo\MyRepos\challenge-collecting-data\data\contents/
C:\Users\ecebo\MyRepos\challenge-collecting-data\data\contents/Contents_9528614.txt
Starting thread  2
https://www.immoweb.be/en/classified/house/for-sale/denderleeuw/9470/9528614

C:\Users\ecebo\MyRepos\challenge-collecting-data\data\contents/
C:\Users\ecebo\MyRepos\challenge-collecting-data\data\contents/Contents_9555271.txt
Starting thread  3
https://www.immoweb.be/en/classified/duplex/for-sale/geraardsbergen/9500/9555271

C:\Users\ecebo\MyRepos\challenge-collecting-data\data\contents/
C:\Users\ecebo\MyRepos\challenge-collecting-data\data\contents/Contents_9064209.txt
Starting thread  4
https://www.immoweb.be/en/classified/house/for-sale/uccle/1180/9064209

C:\

Made request
https://www.immoweb.be/en/classified/apartment/for-sale/ixelles/1050/8956021
 200
160249
Written file ...8956021
C:\Users\ecebo\MyRepos\challenge-collecting-data\data\contents/
C:\Users\ecebo\MyRepos\challenge-collecting-data\data\contents/Contents_9511387.txt
https://www.immoweb.be/en/classified/house/for-sale/isnes/5032/9511387

Made request
https://www.immoweb.be/en/classified/duplex/for-sale/geraardsbergen/9500/9555271
 200
166041
Written file ...9555271
C:\Users\ecebo\MyRepos\challenge-collecting-data\data\contents/
C:\Users\ecebo\MyRepos\challenge-collecting-data\data\contents/Contents_9442043.txt
https://www.immoweb.be/en/classified/house/for-sale/hermalle-sous-huy/4480/9442043

Made request
https://www.immoweb.be/en/classified/villa/for-sale/nivelles/1400/9451348
 200
177112
Written file ...9451348
C:\Users\ecebo\MyRepos\challenge-collecting-data\data\contents/
C:\Users\ecebo\MyRepos\challenge-collecting-data\data\contents/Contents_9374996.txt
https://www.immoweb.b

Made request
https://www.immoweb.be/en/classified/duplex/for-sale/liege-1/4000/8549833
 200
163860
Written file ...8549833
C:\Users\ecebo\MyRepos\challenge-collecting-data\data\contents/
C:\Users\ecebo\MyRepos\challenge-collecting-data\data\contents/Contents_9410683.txt
https://www.immoweb.be/en/classified/apartment/for-sale/bruxelles-ville/1000/9410683

Made request
https://www.immoweb.be/en/classified/apartment/for-sale/bruxelles/1000/9015936
 200
177534
Written file ...9015936
C:\Users\ecebo\MyRepos\challenge-collecting-data\data\contents/
C:\Users\ecebo\MyRepos\challenge-collecting-data\data\contents/Contents_9522770.txt
https://www.immoweb.be/en/classified/apartment/for-sale/ixelles/1050/9522770

Made request
https://www.immoweb.be/en/classified/apartment/for-sale/ixelles/1050/9412877
 200
157424
Written file ...9412877
C:\Users\ecebo\MyRepos\challenge-collecting-data\data\contents/
C:\Users\ecebo\MyRepos\challenge-collecting-data\data\contents/Contents_9419705.txt
https://www.imm

Made request
https://www.immoweb.be/en/classified/apartment/for-sale/willebroek/2830/9419705
 200
159861
Written file ...9419705
C:\Users\ecebo\MyRepos\challenge-collecting-data\data\contents/
C:\Users\ecebo\MyRepos\challenge-collecting-data\data\contents/Contents_9295044.txt
https://www.immoweb.be/en/classified/villa/for-sale/waterloo/1410/9295044

Made request
https://www.immoweb.be/en/classified/villa/for-sale/liege/4000/9444848
 200
170502
Written file ...9444848
C:\Users\ecebo\MyRepos\challenge-collecting-data\data\contents/
C:\Users\ecebo\MyRepos\challenge-collecting-data\data\contents/Contents_9425110.txt
https://www.immoweb.be/en/classified/apartment/for-sale/mont-saint-guibert/1435/9425110

Made request
https://www.immoweb.be/en/classified/house/for-sale/nijlen/2560/9471715
 200
168014
Written file ...9471715
C:\Users\ecebo\MyRepos\challenge-collecting-data\data\contents/
C:\Users\ecebo\MyRepos\challenge-collecting-data\data\contents/Contents_9548314.txt
https://www.immoweb.be

Made request
https://www.immoweb.be/en/classified/bungalow/for-sale/ninove/9400/9334471
 200
181383
Written file ...9334471
C:\Users\ecebo\MyRepos\challenge-collecting-data\data\contents/
C:\Users\ecebo\MyRepos\challenge-collecting-data\data\contents/Contents_9557147.txt
https://www.immoweb.be/en/classified/house/for-sale/farciennes-pironchamps/6240/9557147

Made request
https://www.immoweb.be/en/classified/ground-floor/for-sale/westkapelle/8300/8762092
 200
160817
Written file ...8762092
C:\Users\ecebo\MyRepos\challenge-collecting-data\data\contents/
C:\Users\ecebo\MyRepos\challenge-collecting-data\data\contents/Contents_9413764.txt
https://www.immoweb.be/en/classified/apartment/for-sale/knokke-heist/8301/9413764

Made request
https://www.immoweb.be/en/classified/house/for-sale/mechelen/2800/9550205
 200
168929
Written file ...9550205
C:\Users\ecebo\MyRepos\challenge-collecting-data\data\contents/
C:\Users\ecebo\MyRepos\challenge-collecting-data\data\contents/Contents_9370923.txt
http

Made request
https://www.immoweb.be/en/classified/apartment/for-sale/evere/1140/9547335
 200
158143
Written file ...9547335
C:\Users\ecebo\MyRepos\challenge-collecting-data\data\contents/
C:\Users\ecebo\MyRepos\challenge-collecting-data\data\contents/Contents_9540572.txt
https://www.immoweb.be/en/classified/penthouse/for-sale/deurne/2100/9540572

Made request
https://www.immoweb.be/en/classified/mansion/for-sale/wielsbeke/8710/9509269
 200
164491
Written file ...9509269
C:\Users\ecebo\MyRepos\challenge-collecting-data\data\contents/
C:\Users\ecebo\MyRepos\challenge-collecting-data\data\contents/Contents_9496930.txt
https://www.immoweb.be/en/classified/house/for-sale/gosselies/6041/9496930



In [20]:
# Wait until the queue has been processed
q.join()
print('All tasks completed.')

All tasks completed.


Find the clickable for Search on the list and click on it

In [29]:
r = requests.get(urls[0])
# Display the requested url and the return of the server
print(urls[0], r.status_code)
# Store the HTML content of the website in a text
page_contents = r.content

https://www.immoweb.be/en/classified/apartment/for-sale/ixelles/1050/8956021
 200


In [30]:
print(type(page_contents))
print(page_contents)

<class 'bytes'>
b'<!doctype html>\n\n<html itemscope itemtype="http://schema.org/WebPage" lang="en">\n\n<head>\n    <meta charset="utf-8">\n\n    \n    \n    <script>\n    window.dataLayer = [\n        {\n            "user": {\n                "loginStatus": "logged out",\n                "id": "",\n                "personal": {\n                    "language": ""\n                }\n            },\n                                        "classified": {\n                "id": "8956021",\n                "type": "apartment",\n                "subtype": "apartment",\n                "price": "1150000",\n                "transactionType": "for sale",\n                "zip": "1050",\n                "visualisationOption": "xl",\n                "kitchen": {\n                    "type": "installed"\n                },\n                "building": {\n                    "constructionYear": "1930",\n                    "condition": "good"\n                },\n                "energy": {\n   